In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install pyngrok


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np
import pandas as pd
import requests

def fetch_poster(movie_id):
  response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=fac3ac674fbf79770d00576bcfe5f210'.format(movie_id))
  data= response.json()
  poster_path = data['poster_path']
  return "https://image.tmdb.org/t/p/w500/"+poster_path

def recommand(movie):
  movie_index = movies[movies['title']==movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
  recommended_movies = []
  recommended_movies_posters = []
  for i in movies_list:
    movie_id = movies.iloc[i[0]].id
    recommended_movies.append(movies.iloc[i[0]].title)
    recommended_movies_posters.append(fetch_poster(movie_id))
  return recommended_movies,recommended_movies_posters

movies_list = pickle.load(open('/content/drive/MyDrive/Movie Recommander System Files/movies.pkl','rb'))
movies = pd.DataFrame(movies_list)
movies_list = movies_list['title'].values

similarity = pickle.load(open('/content/drive/MyDrive/Movie Recommander System Files/similarity.pkl','rb'))

st.header("Movie Recommendation System")

selected_movie_name = st.selectbox(
    'Type or select a movie?',
    movies_list)

if st.button('Recommend'):
    names,posters = recommand(selected_movie_name)
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.text(names[0])
        st.image(posters[0])
    with col2:
        st.text(names[1])
        st.image(posters[1])

    with col3:
        st.text(names[2])
        st.image(posters[2])
    with col4:
        st.text(names[3])
        st.image(posters[3])
    with col5:
        st.text(names[4])
        st.image(posters[4])



Overwriting app.py


In [38]:
%%writefile app.py

# trying different UI
import streamlit as st
import pickle
import numpy as np
import pandas as pd
import requests

def fetch_poster(movie_id):
  response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=fac3ac674fbf79770d00576bcfe5f210'.format(movie_id))
  data= response.json()
  return data
  # poster_path = data['poster_path']
  # return "https://image.tmdb.org/t/p/w500/"+poster_path

def movie_info(id):
  response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=fac3ac674fbf79770d00576bcfe5f210'.format(id))
  data= response.json()
  return data

def cast_info(id):
  response = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key=fac3ac674fbf79770d00576bcfe5f210'.format(id))
  data= response.json()
  return data

def recommand(movie):
  movie_index = movies[movies['title']==movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
  recommended_movies = []
  recommended_movies_posters = []
  for i in movies_list:
    movie_id = movies.iloc[i[0]].id
    recommended_movies.append(movies.iloc[i[0]].title)
    movie_info = fetch_poster(movie_id)
    recommended_movies_posters.append("https://image.tmdb.org/t/p/w500/"+movie_info['poster_path'])
  return recommended_movies,recommended_movies_posters

movies_list = pickle.load(open('/content/drive/MyDrive/Movie Recommander System Files/movies.pkl','rb'))
movies = pd.DataFrame(movies_list)
movies_list = movies_list['title'].values

similarity = pickle.load(open('/content/drive/MyDrive/Movie Recommander System Files/similarity.pkl','rb'))

st.header("Movie Recommendation System")

selected_movie_name = st.selectbox(
    'Type or select a movie?',
    movies_list)

if st.button('Recommend'):
    movie_id = movies.loc[movies['title'] == selected_movie_name, 'id'].values[0]
    info = movie_info(movie_id)
    st.markdown(f"<h1 style='text-align: center;'>{info['title']}</h1>", unsafe_allow_html=True)
    # Create two columns
    col1, col2 = st.columns(2)

    # Column 1: Poster
    with col1:
      st.image('https://image.tmdb.org/t/p/w500/' + info['poster_path'])

    # Column 2: Movie details
    with col2:
      genre_names = [genre['name'] for genre in info['genres']]
      st.write(f"**Original Title:** {info['original_title']}")
      st.markdown(
    f"""
    <style>
    .justified-text {{
        text-align: justify;
    }}
    </style>
    <div class="justified-text">
    <strong>Overview:</strong> {info['overview']}
    </div>
    """,
    unsafe_allow_html=True
    )

      st.write(f"**Rating:** {info['vote_average']}/10({info['vote_count']} votes)")
      st.write(f"**Release Date:** {info['release_date']}")
      st.write(f"**Genres:** {', '.join(genre_names)}")
      st.write(f"**Runtime:** {info['runtime']} min")
    st.markdown(f"<h1 style='text-align: center;'>Cast</h1>", unsafe_allow_html=True)
    cast = cast_info(movie_id)
    cast_info = [
    {
        "name": cast_member["name"],
        "character": cast_member["character"],
        "profile_path": 'https://image.tmdb.org/t/p/w185/'+cast_member["profile_path"]
    }
    for cast_member in cast['cast'] if cast_member["profile_path"]
    ]
    # Display top 5 cast members in a row
    cols = st.columns(5)
    for i in range(min(5, len(cast_info))):
        cast_member = cast_info[i]
        with cols[i]:
            st.image(cast_member["profile_path"])
            st.write(f"**Name:** {cast_member['name']}")
            st.write(f"**Character:** {cast_member['character']}")
    st.header("Movies you may also like")
    names,posters = recommand(selected_movie_name)
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.image(posters[0])
        st.write(names[0])

    with col2:
        st.image(posters[1])
        st.write(names[1])
    with col3:
        st.image(posters[2])
        st.write(names[2])

    with col4:
        st.image(posters[3])
        st.write(names[3])

    with col5:
        st.image(posters[4])
        st.write(names[4])




Overwriting app.py


In [39]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.234.174.249:8501

npx: installed 22 in 2.295s
your url is: https://clever-pans-behave.loca.lt
  Stopping...
^C


In [6]:
!curl ipecho.net/plain

35.234.174.249